In [1]:
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import warnings
warnings.filterwarnings(action='ignore')
import numpy as np
plt.rc('font', family='Malgun Gothic') 
plt.rc('axes', unicode_minus=False)
import matplotlib.font_manager as fm
font_path = 'C:\Windows\Fonts\HMFMPYUN.TTF'
font_prop = fm.FontProperties(fname=font_path, size=20)
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, GRU, Dropout
from tensorflow.keras import models
import seaborn as sns
import tensorflow.keras.utils as utils
from tensorflow.keras.callbacks import Callback

In [2]:
seoul_2016=pd.read_csv('data/2016년 서울시 교통사고.csv', encoding='cp949')
seoul_2017=pd.read_csv('data/2017년 서울시 교통사고.csv', encoding='cp949')
seoul_2018=pd.read_csv('data/2018년 서울시 교통사고.csv', encoding='cp949')
seoul_2019=pd.read_csv('data/2019년 서울시 교통사고.csv', encoding='cp949')
seoul_2020=pd.read_csv('data/2020년 서울시 교통사고.csv', encoding='cp949')

In [3]:
seoul=pd.concat([seoul_2016,seoul_2017,seoul_2018,seoul_2019,seoul_2020])

In [4]:
seoul.index=range(seoul.shape[0])

In [5]:
seoul.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 191944 entries, 0 to 191943
Data columns (total 22 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   사고번호        191944 non-null  int64 
 1   사고일시        191944 non-null  object
 2   요일          191944 non-null  object
 3   시군구         191944 non-null  object
 4   사고내용        191944 non-null  object
 5   사망자수        191944 non-null  int64 
 6   중상자수        191944 non-null  int64 
 7   경상자수        191944 non-null  int64 
 8   부상신고자수      191944 non-null  int64 
 9   사고유형        191944 non-null  object
 10  법규위반        191944 non-null  object
 11  노면상태        191944 non-null  object
 12  기상상태        191944 non-null  object
 13  도로형태        191944 non-null  object
 14  가해운전자 차종    191944 non-null  object
 15  가해운전자 성별    191944 non-null  object
 16  가해운전자 연령    191944 non-null  object
 17  가해운전자 상해정도  191944 non-null  object
 18  피해운전자 차종    185992 non-null  object
 19  피해운전자 성별    185992 non-

In [ ]:
seoul.columns

In [ ]:
seoul[seoul.columns[0]].value_counts()

In [ ]:
seoul[seoul.columns[1]].value_counts()

In [ ]:
seoul[seoul.columns[2]].value_counts()

In [ ]:
seoul[seoul.columns[3]].value_counts()

In [ ]:
seoul[seoul.columns[4]].value_counts()

In [ ]:
seoul[seoul.columns[5]].value_counts()

In [ ]:
seoul[seoul.columns[6]].value_counts()

In [ ]:
seoul[seoul.columns[7]].value_counts()

In [ ]:
seoul[seoul.columns[8]].value_counts()

In [ ]:
seoul[seoul.columns[9]].value_counts()

In [ ]:
seoul[seoul.columns[10]].value_counts()

In [ ]:
seoul[seoul.columns[11]].value_counts()

In [ ]:
seoul[seoul.columns[12]].value_counts()

In [ ]:
seoul[seoul.columns[13]].value_counts()

In [ ]:
seoul[seoul.columns[14]].value_counts()

In [ ]:
seoul[seoul.columns[15]].value_counts()

In [ ]:
seoul[seoul.columns[16]].value_counts()

In [ ]:
seoul[seoul.columns[17]].value_counts()

In [ ]:
seoul[seoul.columns[18]].value_counts()

In [ ]:
seoul[seoul.columns[19]].value_counts()

In [ ]:
seoul[seoul.columns[20]].value_counts()

In [ ]:
seoul[seoul.columns[21]].value_counts()

In [ ]:
seoul[seoul.columns[3]].value_counts().index[0]

In [6]:
yeoksam=seoul[seoul[seoul.columns[3]]==seoul[seoul.columns[3]].value_counts().index[0]]['사고번호']

In [7]:
yeoksam.index=range(yeoksam.shape[0])

In [8]:
int(yeoksam[0]/100000000)

20160101

In [9]:
yeoksam=yeoksam.map(lambda x : int(x/100000000))

In [10]:
yeoksam_num=yeoksam.to_frame().groupby('사고번호')['사고번호'].count().to_frame()

In [11]:
yeoksam_num=yeoksam_num['사고번호']

In [12]:
max_yeoksam_num=max(yeoksam_num)
min_yeoksam_num=min(yeoksam_num)

In [13]:
yeoksam_num

사고번호
20160101    3
20160102    2
20160103    1
20160104    5
20160105    1
           ..
20201227    1
20201228    3
20201229    1
20201230    3
20201231    4
Name: 사고번호, Length: 1566, dtype: int64

In [14]:
def seq2dataset(num, window_size):
    dataset = []
    for i in range(len(num) - window_size):
        subset = num[i: (i + window_size + 1)]
        dataset.append(subset)
    return np.array(dataset)

In [15]:
dataset=seq2dataset(yeoksam_num,40)

In [16]:
x_train = dataset[:,:-1]
y_train = dataset[:,-1]
x_train.shape, y_train.shape

((1526, 40), (1526,))

In [17]:
dataset

array([[3, 2, 1, ..., 1, 4, 2],
       [2, 1, 5, ..., 4, 2, 1],
       [1, 5, 1, ..., 2, 1, 1],
       ...,
       [3, 2, 1, ..., 1, 3, 1],
       [2, 1, 4, ..., 3, 1, 3],
       [1, 4, 3, ..., 1, 3, 4]], dtype=int64)

In [18]:
x_train = (x_train-min_yeoksam_num)/(max_yeoksam_num-min_yeoksam_num)
x_train[:2]

array([[0.25 , 0.125, 0.   , 0.5  , 0.   , 0.   , 0.375, 0.25 , 0.25 ,
        0.   , 0.   , 0.375, 0.125, 0.125, 0.   , 0.   , 0.25 , 0.125,
        0.125, 0.125, 0.125, 0.25 , 0.375, 0.125, 0.   , 0.25 , 0.125,
        0.25 , 0.   , 0.375, 0.25 , 0.125, 0.125, 0.   , 0.   , 0.125,
        0.125, 0.   , 0.   , 0.375],
       [0.125, 0.   , 0.5  , 0.   , 0.   , 0.375, 0.25 , 0.25 , 0.   ,
        0.   , 0.375, 0.125, 0.125, 0.   , 0.   , 0.25 , 0.125, 0.125,
        0.125, 0.125, 0.25 , 0.375, 0.125, 0.   , 0.25 , 0.125, 0.25 ,
        0.   , 0.375, 0.25 , 0.125, 0.125, 0.   , 0.   , 0.125, 0.125,
        0.   , 0.   , 0.375, 0.125]])

In [19]:
y_train=y_train-1
y_train = utils.to_categorical(y_train)
y_train.shape

(1526, 9)

In [20]:
x_train.shape

(1526, 40)

In [21]:
x_train=x_train.reshape(1526,40,1)

In [22]:
one_hot_vec_size = y_train.shape[1]

In [ ]:
# 1. 모델 생성
model = Sequential()
model.add(LSTM(
    units=128,
    kernel_initializer='glorot_normal',
    bias_initializer='zero',
    batch_input_shape=(1, 40, 1), 
    stateful=True
))
model.add(Dense(
    units=one_hot_vec_size, 
    kernel_initializer='glorot_normal',
    bias_initializer='zero',    
    activation='softmax'
))
# 2. 모델 학습과정 설정
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)
# 3. 모델 학습시키기
class LossHistory(Callback):
    def __init__(self):
        self.epoch = 0
        self.losses = []
        self.accuracy = []
    def on_epoch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.accuracy.append(logs.get('accuracy'))
        if self.epoch % 100 == 0:
            print("epoch: {0} - loss: {1:8.6f}, acc : {2:8.6f}".\
                  format(self.epoch, logs.get('loss'), logs.get('accuracy')))
        self.epoch += 1
history = LossHistory()
for epoch in range(4000):
    model.fit(x_train, y_train, epochs=1, batch_size=1, verbose=0, shuffle=False,
             callbacks=[history])
    if (history.losses[-1] < 1e-5) & (history.accuracy[-1] > 0.99):
        print("epoch:{} - loss:{}, acc:{} - 종료".\
              format(epoch, history.losses[-1], history.accuracy[-1]))
        break;
    model.reset_states() # 훈련할 때마다 이전 데이터와 상관없이 다른 데이터라고 판단

In [ ]:
# 1. 모델 생성 best!!!!!!!!!!!!!!!!!!
model = Sequential()
model.add(LSTM(
    units=128,
    batch_input_shape=(1, 40, 1), 
    stateful=True
))
model.add(Dense(
    units=one_hot_vec_size,     
    activation='softmax'
))
# 2. 모델 학습과정 설정
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)
# 3. 모델 학습시키기
class LossHistory(Callback):
    def __init__(self):
        self.epoch = 0
        self.losses = []
        self.accuracy = []
    def on_epoch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.accuracy.append(logs.get('accuracy'))
        if self.epoch % 100 == 0:
            print("epoch: {0} - loss: {1:8.6f}, acc : {2:8.6f}".\
                  format(self.epoch, logs.get('loss'), logs.get('accuracy')))
        self.epoch += 1
history = LossHistory()
for epoch in range(4000):
    model.fit(x_train, y_train, epochs=1, batch_size=1, verbose=0, shuffle=False,
             callbacks=[history])
    if (history.losses[-1] < 1e-5) & (history.accuracy[-1] > 0.99):
        print("epoch:{} - loss:{}, acc:{} - 종료".\
              format(epoch, history.losses[-1], history.accuracy[-1]))
        break;
    model.reset_states() # 훈련할 때마다 이전 데이터와 상관없이 다른 데이터라고 판단

In [23]:
# 1. 모델 생성 best!!!!!!!!!!!!!!!!!!
model = Sequential()
model.add(GRU(
    units=128,
    batch_input_shape=(1, 40, 1), 
    stateful=True
))
model.add(Dense(
    units=one_hot_vec_size,     
    activation='softmax'
))
# 2. 모델 학습과정 설정
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)
# 3. 모델 학습시키기
class LossHistory(Callback):
    def __init__(self):
        self.epoch = 0
        self.losses = []
        self.accuracy = []
    def on_epoch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.accuracy.append(logs.get('accuracy'))
        if self.epoch % 100 == 0:
            print("epoch: {0} - loss: {1:8.6f}, acc : {2:8.6f}".\
                  format(self.epoch, logs.get('loss'), logs.get('accuracy')))
        self.epoch += 1
history = LossHistory()
for epoch in range(3000):
    model.fit(x_train, y_train, epochs=1, batch_size=1, verbose=0, shuffle=False,
             callbacks=[history])
    if (history.losses[-1] < 1e-5) & (history.accuracy[-1] > 0.99):
        print("epoch:{} - loss:{}, acc:{} - 종료".\
              format(epoch, history.losses[-1], history.accuracy[-1]))
        break;
    model.reset_states() # 훈련할 때마다 이전 데이터와 상관없이 다른 데이터라고 판단

epoch: 0 - loss: 1.656108, acc : 0.277851
epoch: 100 - loss: 0.711185, acc : 0.726737
epoch: 200 - loss: 0.297887, acc : 0.898427
epoch: 300 - loss: 1.368623, acc : 0.408257
epoch: 400 - loss: 1.291767, acc : 0.470511
epoch: 500 - loss: 1.599805, acc : 0.313893
epoch: 600 - loss: 1.643855, acc : 0.296855
epoch: 700 - loss: 1.534199, acc : 0.336173
epoch: 800 - loss: 1.586363, acc : 0.332241
epoch: 900 - loss: 1.520315, acc : 0.357143
epoch: 1000 - loss: 1.553114, acc : 0.341415
epoch: 1100 - loss: 1.499912, acc : 0.385321


KeyboardInterrupt: 

In [ ]:
seq_in = [0.25]
seq_out = seq_in
seq_in = (seq_in-min_yeoksam_num)/(max_yeoksam_num-min_yeoksam_num)
for i in range(1562):
    sample_in = np.array(seq_in)
    sample_in = np.reshape(sample_in, (1, 4, 1)) # 샘플 수, 타입스텝 수, 속성 수
    pred_out = model.predict(sample_in)
    idx = np.argmax(pred_out)+1
    np.append(seq_out, idx)
    np.append(seq_in, (idx -min_yeoksam_num)/(max_yeoksam_num-min_yeoksam_num))
    seq_in.pop(0)

model.reset_states()

In [ ]:
x_train[0]

In [ ]:
seq_in = [3,2,1,5]
seq_out = seq_in
seq_in = [ (it-min_yeoksam_num)/(max_yeoksam_num-min_yeoksam_num) for it in seq_in] 
for i in range(1562):
    sample_in = np.array(seq_in)
    sample_in = np.reshape(sample_in, (1, 4, 1)) 
    pred_out = model.predict(sample_in)
    idx = np.argmax(pred_out)+1
    seq_out.append(idx)
    seq_in.append((idx-min_yeoksam_num)/(max_yeoksam_num-min_yeoksam_num))
    seq_in.pop(0)

model.reset_states()

In [ ]:
plt.plot(np.array(seq_out[0:100]))

In [ ]:
plt.plot(np.array(yeoksam_num)[0:100])

In [ ]:
plt.plot(np.array(seq_out[0:50]))

In [ ]:
(y_train.argmax(axis=1)+1) # 1 1 4 3 3 1

In [ ]:
np.array(yeoksam_num)[0:30]

In [ ]:
np.array(seq_out[0:30])

In [ ]:
np.array(seq_out[0:50])

In [ ]:
#model.save('yeoksammodel.h5') #50개

In [ ]:
model=models.load_model('yeoksammodel.h5')